In [68]:
!hadoop fs -ls /public/retail_db/customers

Found 1 items
-rw-r--r--   2 hdfs supergroup     953719 2021-01-28 08:59 /public/retail_db/customers/part-00000


In [69]:
!hadoop fs -head /public/retail_db/customers/part-00000

1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725
4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069
5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725
6,Mary,Smith,XXXXXXXXX,XXXXXXXXX,3151 Sleepy Quail Promenade,Passaic,NJ,07055
7,Melissa,Wilcox,XXXXXXXXX,XXXXXXXXX,9453 High Concession,Caguas,PR,00725
8,Megan,Smith,XXXXXXXXX,XXXXXXXXX,3047 Foggy Forest Plaza,Lawrence,MA,01841
9,Mary,Perez,XXXXXXXXX,XXXXXXXXX,3616 Quaking Street,Caguas,PR,00725
10,Melissa,Smith,XXXXXXXXX,XXXXXXXXX,8598 Harvest Beacon Plaza,Stafford,VA,22554
11,Mary,Huffman,XXXXXXXXX,XXXXXXXXX,3169 Stony Woods,Caguas,PR,00725
12,Christopher,Smith,XXXXXXXXX,XXXXXXXXX,5594 Jagged Embers By-pass,San Antonio,TX,78227
13,Mary,Baldwin,XXXXXXXXX,XXXXXXXXX,7922 Iron Oak Gardens,Caguas,PR,00725
14,Katherine

## External Table Example

In [10]:
import getpass as gp
from pyspark.sql import SparkSession, functions as F

In [11]:
user = gp.getuser()
spark = SparkSession.builder \
            .appName(f'{user}-external-table-example') \
            .master('yarn') \
            .config('spark.sql.warehouse.dir', f'/user/{user}/warehouse') \
            .config('spark.sql.catalogImplementation', 'hive') \
            .enableHiveSupport() \
            .getOrCreate()

In [12]:
spark

In [15]:
spark.sql("show databases").filter(f'namespace like "%{user}%"').show()

+---------+
|namespace|
+---------+
+---------+



In [19]:
spark.sql(f"create database if not exists {user}_database")

""


In [20]:
spark.sql("show databases").filter(f'namespace like "%{user}%"').show()

+------------------+
|         namespace|
+------------------+
|itv005077_database|
+------------------+



In [21]:
spark.sql(f'use {user}_database')

""


In [70]:
spark.sql("show tables")

database,tableName,isTemporary


In [71]:
spark.sql(f"create table {user}_database.customers \
          ( \
              cust_id integer, \
              first_name string, \
              last_name string, \
              email string, \
              phone string, \
              address_line string, \
              city string, \
              state string, \
              zip integer \
            ) \
        using csv \
        location '/public/retail_db/customers'")

""


In [72]:
spark.sql("show tables")

database,tableName,isTemporary
itv005077_database,customers,false


In [73]:
spark.sql(f"describe extended {user}_database.customers").show(truncate= False)

+----------------------------+--------------------------------------------------------+-------+
|col_name                    |data_type                                               |comment|
+----------------------------+--------------------------------------------------------+-------+
|cust_id                     |int                                                     |null   |
|first_name                  |string                                                  |null   |
|last_name                   |string                                                  |null   |
|email                       |string                                                  |null   |
|phone                       |string                                                  |null   |
|address_line                |string                                                  |null   |
|city                        |string                                                  |null   |
|state                       |string    

In [74]:
spark.sql("select * from customers").limit(10).show()

+-------+----------+---------+---------+---------+--------------------+-----------+-----+-----+
|cust_id|first_name|last_name|    email|    phone|        address_line|       city|state|  zip|
+-------+----------+---------+---------+---------+--------------------+-----------+-----+-----+
|      1|   Richard|Hernandez|XXXXXXXXX|XXXXXXXXX|  6303 Heather Plaza|Brownsville|   TX|78521|
|      2|      Mary|  Barrett|XXXXXXXXX|XXXXXXXXX|9526 Noble Embers...|  Littleton|   CO|80126|
|      3|       Ann|    Smith|XXXXXXXXX|XXXXXXXXX|3422 Blue Pioneer...|     Caguas|   PR|  725|
|      4|      Mary|    Jones|XXXXXXXXX|XXXXXXXXX|  8324 Little Common| San Marcos|   CA|92069|
|      5|    Robert|   Hudson|XXXXXXXXX|XXXXXXXXX|10 Crystal River ...|     Caguas|   PR|  725|
|      6|      Mary|    Smith|XXXXXXXXX|XXXXXXXXX|3151 Sleepy Quail...|    Passaic|   NJ| 7055|
|      7|   Melissa|   Wilcox|XXXXXXXXX|XXXXXXXXX|9453 High Concession|     Caguas|   PR|  725|
|      8|     Megan|    Smith|XXXXXXXXX|

In [76]:
!hadoop fs -ls warehouse

Found 1 items
drwxr-xr-x   - itv005077 supergroup          0 2023-05-16 13:53 warehouse/itv005077_database.db


In [77]:
!hadoop fs -ls warehouse/itv005077_database.db

In [78]:
spark.sql("drop table customers")

""


In [79]:
!hadoop fs -ls /public/retail_db/customers

Found 1 items
-rw-r--r--   2 hdfs supergroup     953719 2021-01-28 08:59 /public/retail_db/customers/part-00000


In [81]:
spark.sql("drop database itv005077_database")

""


In [82]:
!hadoop fs -ls warehouse

In [83]:
!hadoop fs -ls

Found 3 items
drwx------   - itv005077 supergroup          0 2023-04-15 10:10 .Trash
drwxr-xr-x   - itv005077 supergroup          0 2023-05-16 13:18 .sparkStaging
drwxr-xr-x   - itv005077 supergroup          0 2023-05-16 13:56 warehouse


In [84]:
spark.stop()